（副読本）
* ニールセン・チャン「量子コンピュータと量子通信」（ここでは N,C"QCQI" と書きます）
* arXivに公開されている論文

In [1]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import Qubit, QubitBra, measure_all, measure_all_oneshot,measure_partial, matrix_to_qubit
from sympy.physics.quantum.gate import H,X,Y,Z,S,T,CPHASE,CNOT,SWAP,UGate,CGateS,gate_simp,OneQubitGate
from sympy.physics.quantum.gate import IdentityGate as _I
%matplotlib inline
import matplotlib.pyplot as plt
from sympy.physics.quantum.circuitplot import CircuitPlot,labeller, Mz,CreateOneQubitGate
def CCX(c1,c2,t):  return CGateS((c1,c2),X(t))

# 6. 量子コンピュータのエラー訂正

## 6.5 トポロジカル符号は奇妙なアイディア？

### 6.5.1 トーラスを”ひらめき”にする（表面符号）

[Fault-tolerant quantum computation by anyons(arXiv:quant-ph/9707021)](https://arxiv.org/abs/quant-ph/9707021)

Keywords: 　トポロジカル符号、 KitaevのToric Code、 表面符号

### 6.5.2 ５量子ビットの表面符号

表面符号の基本形


In [2]:
L0 =1/2*( Qubit("00000")+Qubit("00111")+Qubit("11011")+Qubit("11100"))
L1=1/2*( Qubit("01001")+Qubit("01110")+Qubit("10010")+Qubit("10101"))

In [3]:
S1=X(0)*X(1)*X(2)
S2=X(2)*X(3)*X(4)
S3=Z(0)*Z(2)*Z(3)
S4=Z(1)*Z(2)*Z(4)

In [4]:
qapply(S1*L0)

0.5*|00000> + 0.5*|00111> + 0.5*|11011> + 0.5*|11100>

In [5]:
qapply(S2*L0)

0.5*|00000> + 0.5*|00111> + 0.5*|11011> + 0.5*|11100>

In [6]:
qapply(S3*L0)

0.5*|00000> + 0.5*|00111> + 0.5*|11011> + 0.5*|11100>

In [7]:
qapply(S4*L0)

0.5*|00000> + 0.5*|00111> + 0.5*|11011> + 0.5*|11100>

In [8]:
qapply(S1*L1)

0.5*|01001> + 0.5*|01110> + 0.5*|10010> + 0.5*|10101>

In [9]:
qapply(S2*L1)

0.5*|01001> + 0.5*|01110> + 0.5*|10010> + 0.5*|10101>

In [10]:
qapply(S3*L1)

0.5*|01001> + 0.5*|01110> + 0.5*|10010> + 0.5*|10101>

In [11]:
qapply(S4*L1)

0.5*|01001> + 0.5*|01110> + 0.5*|10010> + 0.5*|10101>

In [12]:
errX3=X(2)*L0
qapply(errX3)

0.5*|00011> + 0.5*|00100> + 0.5*|11000> + 0.5*|11111>

In [13]:
qapply(S1*errX3)

0.5*|00011> + 0.5*|00100> + 0.5*|11000> + 0.5*|11111>

In [14]:
qapply(S2*errX3)

0.5*|00011> + 0.5*|00100> + 0.5*|11000> + 0.5*|11111>

In [15]:
qapply(S3*errX3)

-0.5*|00011> - 0.5*|00100> - 0.5*|11000> - 0.5*|11111>

In [16]:
qapply(S4*errX3)

-0.5*|00011> - 0.5*|00100> - 0.5*|11000> - 0.5*|11111>

### 6.5.3 誤り耐性を大きくするには？

複数の誤りに対して耐性をもたせるようにするには、訂正するべき量子ビットが十分離れていなければ訂正できなさそう


### 6.5.4 しきい値定理と誤り耐性量子計算

---

【重要な定理】しきい値定理　

各論理ゲートの雑音がある一定のしきい値以下ならば、任意の大きな量子計算を効率的く行うことができる
（N,C"QCQI"の１０章を参照のこと）

↓言い換え

量子ゲート操作のエラー率があるしきい値より小さければ、任意精度の量子計算を多項式時間で実行できる
（テキスト）

---

表面符号は、サイズを大きくすれば物理エラーから論理量子ビットを護りやすくなる
⇔ 一方で、サイズが大きくなると物理量子ビットが多くなり物理エラーが増える

物理エラーが 1% 未満であれば、表面符号がうまくいく

<font color="red"> 物理量子ビットのエラー率は 1% 未満がハードウェアの目標となる </font>


## 6.6 論理ゲート操作を作ろう


表面符号のタイプには大きくわけて、２種類のタイプがあり、どちらがより効率がよいか等の検証がされています

* Lattice Surgery　【planar型】

[Surface code quantum computing by lattice surgery(arXiv:1111.4022)](https://arxiv.org/abs/1111.4022)

[A Game of Surface Codes: Large-Scale Quantum Computing with Lattice Surgery(arXiv:1808.02892)](https://arxiv.org/abs/1808.02892)


* Defect Braiding　【double-defect型】

[Surface codes: Towards practical large-scale quantum computation(arXiv:1208.0928)](https://arxiv.org/abs/1208.0928)



### 6.6.1 論理１量子ビットゲート操作

論理量子ゲート操作

論理 X ゲート操作　＝　対象の範囲の左から右まで繋げた線上のすべての物理量子ビットに X ゲート操作をする  
論理 Z ゲート操作　＝　対象の範囲の上から下まで繋げた線上のすべての物理量子ビットに Z ゲート操作をする


1208.0928 Fig1: データ量子ビットは白丸（○）、エラー訂正のために測定する量子ビットは黒丸（●）
<img src="1208.0928_fig1.png" width="400" />

1208.0928 Fig3:論理１量子ビットゲート操作
<img src="1208.0928_fig3.png" width="400" />

### 6.6.2 論理２量子ビットゲート操作（Lattice surgery）



CNOTを考えると、Lattice surgery は論理３量子ビットの切り貼りで実現できます。

（参考） Defect brading は２つの論理量子ビットを動かす（ defect の周りを編む（ braiding する ）　操作で実現します。